In [1]:
import astropy.coordinates as coord
import astropy.units as u
import numpy as np
from astropy.coordinates import BaseCoordinateFrame, Attribute, RepresentationMapping
from astropy.coordinates import frame_transform_graph

In [2]:
def polar2cart(ra,dec):
    
    x = np.cos(np.deg2rad(ra)) * np.cos(np.deg2rad(dec))
    y = np.sin(np.deg2rad(ra)) * np.cos(np.deg2rad(dec))
    z = np.sin(np.deg2rad(dec))
    
    return np.array([x,y,z])



def cart2polar(vector):

    ra = np.arctan2(vector[1],vector[0]) 
    dec = np.arcsin(vector[2])
    
    return np.rad2deg(ra), np.rad2deg(dec)
    
    
def construct_scy(scx_ra, scx_dec, scz_ra, scz_dec):
    
    x = polar2cart(scx_ra, scx_dec)
    z = polar2cart(scz_ra, scz_dec)
    
    return cart2polar(np.cross(x,-z))
    
def construct_sc_matrix(scx_ra, scx_dec, scy_ra, scy_dec, scz_ra, scz_dec):
    
    sc_matrix = np.zeros((3,3))
    
    sc_matrix[0,:] = polar2cart(scx_ra, scx_dec)
    sc_matrix[1,:] = polar2cart(scy_ra, scy_dec)
    sc_matrix[2,:] = polar2cart(scz_ra, scz_dec)
    
    return sc_matrix
    



class SPIFrame(BaseCoordinateFrame):
    """
    
    INTEGRAL SPI Frame
    Parameters
    ----------
    representation : `BaseRepresentation` or None
        A representation object or None to have no data (or use the other keywords)
  
    """
    default_representation = coord.SphericalRepresentation

    frame_specific_representation_info = {
        'spherical': [
            RepresentationMapping(
                reprname='lon', framename='lon', defaultunit=u.degree),
            RepresentationMapping(
                reprname='lat', framename='lat', defaultunit=u.degree),
            RepresentationMapping(
                reprname='distance', framename='DIST', defaultunit=None)
        ],
        'unitspherical': [
            RepresentationMapping(
                reprname='lon', framename='lon', defaultunit=u.degree),
            RepresentationMapping(
                reprname='lat', framename='lat', defaultunit=u.degree)
        ],
        'cartesian': [
            RepresentationMapping(
                reprname='x', framename='SCX'), RepresentationMapping(
                reprname='y', framename='SCY'), RepresentationMapping(
                reprname='z', framename='SCZ')
        ]
    }

    # Specify frame attributes required to fully specify the frame
    scx_ra = Attribute(default=None)
    scx_dec = Attribute(default=None)
    
    scy_ra = Attribute(default=None)
    scy_dec = Attribute(default=None)
    
    scz_ra = Attribute(default=None)
    scz_dec = Attribute(default=None)
   

    # equinox = TimeFrameAttribute(default='J2000')




@frame_transform_graph.transform(coord.FunctionTransform, SPIFrame, coord.ICRS)
def spi_to_j2000(spi_coord, j2000_frame):
    """ 
    
    """

    sc_matrix = construct_sc_matrix(spi_coord.scx_ra,
                                    spi_coord.scx_dec,
                                    spi_coord.scy_ra,
                                    spi_coord.scy_dec,
                                    spi_coord.scz_ra,
                                    spi_coord.scz_dec)

    # X,Y,Z = gbm_coord.cartesian

    pos = spi_coord.cartesian.xyz.value

    X0 = np.dot(sc_matrix[:, 0], pos)
    X1 = np.dot(sc_matrix[:, 1], pos)
    X2 = np.clip(np.dot(sc_matrix[:, 2], pos), -1., 1.)

    dec = np.arcsin(X2)

    idx = np.logical_and(np.abs(X0) < 1E-6, np.abs(X1) < 1E-6)

    ra = np.zeros_like(dec)

    ra[~idx] = np.arctan2(X1, X0) % (2 * np.pi)

    return coord.ICRS(ra=ra * u.radian, dec=dec * u.radian)


@frame_transform_graph.transform(coord.FunctionTransform, coord.ICRS, SPIFrame)
def j2000_to_spi(j2000_frame, spi_coord):
    """ 
    
    """

    sc_matrix = construct_sc_matrix(spi_coord.scx_ra,
                                    spi_coord.scx_dec,
                                    spi_coord.scy_ra,
                                    spi_coord.scy_dec,
                                    spi_coord.scz_ra,
                                    spi_coord.scz_dec)


    pos = j2000_frame.cartesian.xyz.value

    X0 = np.dot(sc_matrix[0, :], pos)
    X1 = np.dot(sc_matrix[1, :], pos)
    X2 = np.dot(sc_matrix[2, :], pos)

    lat = np.pi / 2. - np.arccos(X2)  # convert to proper frame

    idx = np.logical_and(np.abs(X0) < 1E-6, np.abs(X1) < 1E-6)

    lon = np.zeros_like(lat)

    lon[~idx] = np.arctan2(X1, X0) % (2 * np.pi)

    return SPIFrame(
        lon=lon * u.radian,
        lat=lat * u.radian,
        scx_ra=spi_coord.scx_ra,
        scx_dec=spi_coord.scx_dec,
        scy_ra=spi_coord.scy_ra,
        scy_dec=spi_coord.scy_dec,
        scz_ra=spi_coord.scz_ra,
        scz_dec=spi_coord.scz_dec
    
    )

In [ ]:

class SPIFrame(coord.Sp):

In [175]:
from  astropy.coordinates.coordsystems import SphericalCoordinateBase

ImportError: No module named coordsystems

In [176]:
from astropy.coordinates import SphericalCoordinatesBase

ImportError: cannot import name SphericalCoordinatesBase

In [174]:
SphericalRepresentation?

In [203]:
spi_frame = SPIFrame(lat=90.*u.deg,lon=0.*u.deg
                     ,
                    # unit='deg',
                     scx_ra=206.1563262939453,
 scx_dec=50.5438346862793,
 scy_ra=65.40575996001127,
 scy_dec=32.51224513331057,
 scz_ra=142.12562561035156,
 scz_dec=np.zeros((3,3)))

ValueError: attribute scz_dec should be scalar or have shape (), but is has shape (3, 3) and could not be broadcast.

In [152]:
from astropy.coordinates import ICRS

In [153]:
spi_frame.transform_to(ICRS)

<ICRS Coordinate: (ra, dec) in deg
    (142.12562561, -19.81947136)>

In [199]:
import astropy.io.fits as fits

f = fits.open('sc_orbit_param.fits.gz')

def extract_sc_pointing(pointing_extension, time_index):
    
    scx_ra = pointing_extension.data['RA_SCX'][time_index]
    scx_dec = pointing_extension.data['DEC_SCX'][time_index]
    
    scz_ra = pointing_extension.data['RA_SCZ'][time_index]
    scz_dec = pointing_extension.data['DEC_SCZ'][time_index]
    
    scy_ra, scy_dec = construct_scy(scx_ra, scx_dec, scz_ra, scz_dec)
    
    return  scx_ra, scx_dec, scy_ra, scy_dec, scz_ra, scz_dec
    

def polar2cart(ra,dec):
    
    x = np.cos(np.deg2rad(ra)) * np.cos(np.deg2rad(dec))
    y = np.sin(np.deg2rad(ra)) * np.cos(np.deg2rad(dec))
    z = np.sin(np.deg2rad(dec))
    
    return np.array([x,y,z])



def cart2polar(vector):

    r=np.sqrt((np.array(vector)**2).sum())
    
    ra = np.arctan2(vector[1],vector[0])
    
    dec = np.arcsin(vector[2]/r)
    
    return np.rad2deg(ra), np.rad2deg(dec)
    
    
def construct_scy(scx_ra, scx_dec, scz_ra, scz_dec):
    
    x = polar2cart(scx_ra, scx_dec)
    z = polar2cart(scz_ra, scz_dec)
    
    return cart2polar(np.cross(x,-z))
    
def construct_sc_matrix(scx_ra, scx_dec, scy_ra, scy_dec, scz_ra, scz_dec):
    
    sc_matrix = np.zeros((3,3))
    
    sc_matrix[0,:] = polar2cart(scx_ra, scx_dec)
    sc_matrix[1,:] = polar2cart(scy_ra, scy_dec)
    sc_matrix[2,:] = polar2cart(scz_ra, scz_dec)
    
    return sc_matrix
    
    


In [201]:
f.info()

Filename: sc_orbit_param.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      20   ()      
  1  INTL-ORBI-SCP    1 BinTableHDU     90   417R x 16C   [4I, 1I, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D]   


In [131]:
def transform_via_matric(sc_matrix, ra, dec):
    
    
    pos = polar2cart(ra,dec)
    
    X0 = np.dot(sc_matrix[0, :], pos)
    X1 = np.dot(sc_matrix[1, :], pos)
    X2 = np.dot(sc_matrix[2, :], pos)

    lat = np.pi / 2. - np.arccos(X2)  # convert to proper frame

    #idx = np.logical_and(np.abs(X0) < 1E-6, np.abs(X1) < 1E-6)

    #log = np.zeros_like(el)

    lon = np.arctan2(X1, X0) % (2 * np.pi)
    
    
    return np.rad2deg(lon), np.rad2deg(lat)
    
    
        
        
    

In [168]:
from astropy.coordinates import spherical_to_cartesian, cartesian_to_spherical, CartesianRepresentation

In [177]:
a=CartesianRepresentation(1,1,1)
a.

In [195]:
spherical_to_cartesian?

In [188]:
_, lat, lon = cartesian_to_spherical(1,1,1)
print(lon.deg,lat.deg)

45.0 35.2643896828


In [197]:
spherical_to_cartesian(1.,np.deg2rad(30.),np.deg2rad(20.)).value

AttributeError: 'tuple' object has no attribute 'value'

In [198]:
cartesian_to_spherical?

In [138]:
extract_sc_pointing(f[1],0)

(206.1563262939453,
 50.5438346862793,
 65.40575996001127,
 32.51224513331057,
 142.12562561035156,
 -19.81947135925293)

In [124]:
from astropy.coordinates import SkyCoord

source = SkyCoord(ra=197.075, dec=58.98,unit='deg',frame='icrs')
center = SkyCoord(ra=206.1563262939453, dec=50.5438346862793, unit='deg', frame='icrs')

source.separation(center).value



9.911455021418327

In [132]:
sc_mat = construct_sc_matrix(*extract_sc_pointing(f[1],0))

transform_via_matric(sc_mat,ra=197.075, dec=58.98 )

(9.887256967991247, -0.695638114653079)

In [134]:
source = SkyCoord(9.88,-0.6956381146530,unit='deg')
#source = SkyCoord(0, 10,unit='deg')
center = SkyCoord(0, 0,unit='deg',)

source.separation(center).value

9.904216597936284

In [125]:
 transform(*extract_sc_pointing(f[1],0), ra=197.075, dec=58.98)

(9.911455021418313, 355.955294968188)

In [86]:
def cos_dir(sc_ra,sc_dec, ra, dec):
    return np.sin(np.deg2rad(sc_dec)) * np.sin(np.deg2rad(dec)) + np.cos(np.deg2rad(sc_dec)) * np.cos(np.deg2rad(dec)) * np.cos(np.deg2rad(sc_ra- ra) )
    
    

def transform(scx_ra, scx_dec, scy_ra, scy_dec, scz_ra, scz_dec, ra, dec):
    
    
    
    
    cos_theta = cos_dir(scx_ra,scx_dec, ra, dec)
    
    
    cos_theta = np.clip(cos_theta,-1.,1.)
    
    theta = np.rad2deg(np.arccos(cos_theta))
    
    cos_z = cos_dir(scz_ra,scz_dec, ra, dec)
    cos_y = cos_dir(scy_ra,scy_dec, ra, dec)
    
    phi = np.rad2deg(np.arctan2(cos_z, cos_y))
    
    if phi < 0.0:
        phi += 360.
    
    return theta, phi
    
   

    
    

In [ ]:
Attribute()